# Data QA
#### Last Updated: 2019 June 14 

In [5]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from IPython.display import Image

In [ ]:
%matplotlib inline

plt.rcParams['figure.figsize'] = (6,6)

Goal is to associate risk of account with risk of entity.
Behavioral is based on account.
Acceptance risk - see risk tables (XL files).

### Part 1
Columns with 1) entity something and 2) behavioural risk. *Assign continuous risk for entity using acceptance model.* Metrics.

### Part 2
Linear regression and comparative metrics.

### Part 3
Try other regression types (logarithmic, decision tree, etc)

In [6]:
bRisk = pd.read_csv('1_Data Files/behavioural_risk.csv', sep = ";")
bRisk.head()
bRisk.sort_values(by = "account_number").head()

,account_number,continuous_risk,discrete_risk
5536,19510001,0.454530,3
22872,48610001,0.112144,1
29024,58310001,0.085539,1
3613,68010001,0.329948,2
32169,68019006,0.045224,1


In [7]:
accounts = pd.read_csv('1_Data Files/accounts.csv', sep = ";")
accounts.head()
accounts.sort_values(by = "account_number").head()

,client_number,account_number
0,195,19510001
24395,195,19519001
24522,195,19519002
1,292,29210001
2,486,48610001


In [28]:
entities = pd.read_csv('1_Data Files/entities.csv', sep = ";")
entities.head()

,entity_number,entity_type,date_of_birth,qualifications_code,occupation_activity_code,occupation_code,nationality_code,country_of_residence_code,economic_activity_main_code,economic_activity_sec_code,...,country_of_residence_risk,cutoff_cl,economic_activity_code_risk,nationality_risk,occupation_risk,qualifications_risk,reg,score,seg,society_type_risk
0,2,P,19460908,4,90.0,NaN,CPV,CPV,NaN,NaN,...,1,0,NaN,1.0,4.0,4.0,0.241569,0.290509,P,NaN
1,3,P,19391124,4,10.0,NaN,PRT,PRT,NaN,NaN,...,2,0,NaN,2.0,4.0,4.0,0.411046,0.423553,P,NaN
2,4,P,19460101,2,13.0,NaN,PRT,PRT,NaN,NaN,...,2,0,NaN,2.0,2.0,2.0,0.250000,0.268998,P,NaN
3,5,P,19760709,1,90.0,NaN,CPV,CPV,NaN,NaN,...,1,0,NaN,1.0,4.0,3.0,0.226793,0.269547,P,NaN
4,6,P,19700328,3,13.0,NaN,CPV,CPV,NaN,NaN,...,1,0,NaN,1.0,2.0,3.0,0.095298,0.145518,P,NaN


In [9]:
entities.shape

(65535, 36)

In [11]:
clients = pd.read_csv('1_Data Files/entity_client.csv', sep = ";")
clients.head()

,client_number,entity_number,is_first
0,389,4,True
1,486,5,True
2,583,6,True
3,971,8,True
4,1359,15,True


`is_first` means the entity is the main/more powerful of the client
**an entity is a person** This is not a relevant column so we shall delete it.

In [12]:
del clients["is_first"]
clients.dtypes

client_number    int64
entity_number    int64
dtype: object

In [13]:
bRisk.sort_values(by = "account_number").head()

,account_number,continuous_risk,discrete_risk
5536,19510001,0.454530,3
22872,48610001,0.112144,1
29024,58310001,0.085539,1
3613,68010001,0.329948,2
32169,68019006,0.045224,1


In [14]:
risk_per_acc = accounts.merge(bRisk)
risk_per_acc.head()

,client_number,account_number,continuous_risk,discrete_risk
0,195,19510001,0.454530,3
1,486,48610001,0.112144,1
2,583,58310001,0.085539,1
3,680,68010001,0.329948,2
4,777,77710001,0.161343,1


In [15]:
numbers = clients.merge(risk_per_acc)
numbers.head()

,client_number,entity_number,account_number,continuous_risk,discrete_risk
0,486,5,48610001,0.112144,1
1,583,6,58310001,0.085539,1
2,971,8,97110001,0.179198,1
3,971,8,97119005,0.095819,1
4,1359,15,135910001,0.179872,1


## Duplicates

We need to check for duplicates to assure the quality of our data. First we will look at the unique client-entity identifiers, then the client-account numbers.

### Unique entities

In [13]:
clients.client_number.value_counts().head()

73236      16
74303      16
2842489    11
510221     10
1826414    10
Name: client_number, dtype: int64

In [14]:
clients.entity_number.value_counts().head()

25652    22
15078    19
5993     16
1699     16
18560    14
Name: entity_number, dtype: int64

Here we can see that there are repeated entity numbers and repeated client numbers. To make sure there are not two repeated values connected, we will create a `unique_entities` column and check the value counts on those.

In [15]:
clients['unique_entities'] = clients.client_number.astype(str).str.cat(
    [clients.entity_number.astype(str)],sep='-')

In [16]:
clients.unique_entities.value_counts().head()

1255472-15549    1
1922929-23285    1
2266697-27138    1
3916279-45639    1
2184829-26183    1
Name: unique_entities, dtype: int64

The `unique_entities` column has no duplicates.

### Unique Accounts

In [17]:
accounts.client_number.value_counts().head()

6015       12
22311      12
2282993    12
112133     12
3183153    12
Name: client_number, dtype: int64

In [20]:
accounts.account_number.value_counts().head()

510055210001    1
409097610001    1
343826310001    1
607753610001    1
568022410001    1
Name: account_number, dtype: int64

We can see here that all the account numbers are unique, but we shall create a `unique_accounts` column just in case

In [21]:
accounts['unique_accounts'] = accounts.client_number.astype(str).str.cat(
    [accounts.account_number.astype(str)],sep='-')

In [22]:
accounts.unique_accounts.value_counts().head()

423794-42379415002      1
3282093-328209311001    1
2457302-245730210001    1
5928835-592883510001    1
2109945-210994510001    1
Name: unique_accounts, dtype: int64

There are no duplicates! We can delete the unique columns.

In [ ]:
del clients["unique_entities"]
del accounts["unique_accounts"]

 As a final check for duplicates we will make a unique column in the `numbers` dataframe.

In [29]:
numbers['unique_id'] = numbers.client_number.astype(str).str.cat(
    [numbers.account_number.astype(str), numbers.entity_number.astype(str)],sep='-')

In [32]:
numbers.unique_id.value_counts().head()

1751627-175162710001-21315    1
2656637-265663710001-31408    1
367340-36734010001-3562       1
318937-31893710001-4370       1
1958722-195872210001-23689    1
Name: unique_id, dtype: int64

All are unique, there are no duplicates.

Next we will begin wading through the variables in the `entities` dataframe.

In [33]:
entities.head(10)

,entity_number,entity_type,date_of_birth,qualifications_code,occupation_activity_code,occupation_code,nationality_code,country_of_residence_code,economic_activity_main_code,economic_activity_sec_code,...,country_of_residence_risk,cutoff_cl,economic_activity_code_risk,nationality_risk,occupation_risk,qualifications_risk,reg,score,seg,society_type_risk
0,2,P,19460908,4,90.0,NaN,CPV,CPV,NaN,NaN,...,1,0,NaN,1.0,4.0,4.0,0.241569,0.290509,P,NaN
1,3,P,19391124,4,10.0,NaN,PRT,PRT,NaN,NaN,...,2,0,NaN,2.0,4.0,4.0,0.411046,0.423553,P,NaN
2,4,P,19460101,2,13.0,NaN,PRT,PRT,NaN,NaN,...,2,0,NaN,2.0,2.0,2.0,0.250000,0.268998,P,NaN
3,5,P,19760709,1,90.0,NaN,CPV,CPV,NaN,NaN,...,1,0,NaN,1.0,4.0,3.0,0.226793,0.269547,P,NaN
4,6,P,19700328,3,13.0,NaN,CPV,CPV,NaN,NaN,...,1,0,NaN,1.0,2.0,3.0,0.095298,0.145518,P,NaN
5,7,P,19770506,2,13.0,NaN,CPV,CPV,NaN,NaN,...,1,0,NaN,1.0,2.0,2.0,0.080523,0.135954,P,NaN
6,8,P,19680330,4,13.0,NaN,CPV,CPV,NaN,NaN,...,1,0,NaN,1.0,2.0,4.0,0.110073,0.166481,P,NaN
7,9,P,19730620,2,13.0,NaN,CPV,CPV,NaN,NaN,...,1,0,NaN,1.0,2.0,2.0,0.080523,0.135954,P,NaN
8,10,P,19710222,4,13.0,NaN,CPV,CPV,NaN,NaN,...,1,0,NaN,1.0,2.0,4.0,0.110073,0.166481,P,NaN
9,11,P,19720119,4,10.0,NaN,CPV,CPV,NaN,NaN,...,1,0,NaN,1.0,4.0,4.0,0.241569,0.290509,P,NaN


In [36]:
entities.entity_number.value_counts().head()

2047     1
27288    1
35524    1
45763    1
47810    1
Name: entity_number, dtype: int64

All `entity_number`s in `entities` are unique. There are no duplicates.

In [37]:
entities.dtypes

entity_number                      int64
entity_type                       object
date_of_birth                      int64
qualifications_code               object
occupation_activity_code         float64
occupation_code                  float64
nationality_code                  object
country_of_residence_code         object
economic_activity_main_code      float64
economic_activity_sec_code       float64
date_of_constitution               int64
society_type_code                 object
gender                            object
civil_status                      object
BC_reg                             int64
BC_score                           int64
B_age_risk                       float64
B_company_age_risk               float64
B_country_of_residence_risk      float64
B_economic_activity_code_risk    float64
B_nationality_risk               float64
B_occupation_risk                float64
B_qualifications_risk            float64
B_society_type_risk              float64
age_risk        

Delete `seg` because it has the same values as entity_type

In [38]:
del entities["seg"]

What is `cutoff_cl`?

In [40]:
entities.loc[(entities["cutoff_cl"] == 0)].head()

,entity_number,entity_type,date_of_birth,qualifications_code,occupation_activity_code,occupation_code,nationality_code,country_of_residence_code,economic_activity_main_code,economic_activity_sec_code,...,company_age_risk,country_of_residence_risk,cutoff_cl,economic_activity_code_risk,nationality_risk,occupation_risk,qualifications_risk,reg,score,society_type_risk
0,2,P,19460908,4,90.0,NaN,CPV,CPV,NaN,NaN,...,NaN,1,0,NaN,1.0,4.0,4.0,0.241569,0.290509,NaN
1,3,P,19391124,4,10.0,NaN,PRT,PRT,NaN,NaN,...,NaN,2,0,NaN,2.0,4.0,4.0,0.411046,0.423553,NaN
2,4,P,19460101,2,13.0,NaN,PRT,PRT,NaN,NaN,...,NaN,2,0,NaN,2.0,2.0,2.0,0.250000,0.268998,NaN
3,5,P,19760709,1,90.0,NaN,CPV,CPV,NaN,NaN,...,NaN,1,0,NaN,1.0,4.0,3.0,0.226793,0.269547,NaN
4,6,P,19700328,3,13.0,NaN,CPV,CPV,NaN,NaN,...,NaN,1,0,NaN,1.0,2.0,3.0,0.095298,0.145518,NaN


In [41]:
entities.loc[(entities["cutoff_cl"] == 1)].head()

,entity_number,entity_type,date_of_birth,qualifications_code,occupation_activity_code,occupation_code,nationality_code,country_of_residence_code,economic_activity_main_code,economic_activity_sec_code,...,company_age_risk,country_of_residence_risk,cutoff_cl,economic_activity_code_risk,nationality_risk,occupation_risk,qualifications_risk,reg,score,society_type_risk
80,93,E,0,NaN,NaN,NaN,NaN,NaN,8890.0,NaN,...,3.0,3,1,4.0,NaN,NaN,NaN,0.468536,0.445717,1.0
101,120,E,0,NaN,NaN,NaN,NaN,CPV,8890.0,NaN,...,1.0,1,1,4.0,NaN,NaN,NaN,0.480133,0.440667,4.0
111,130,P,19480222,0,0.0,NaN,CUB,CUB,NaN,NaN,...,NaN,5,1,NaN,5.0,5.0,5.0,1.000000,0.971992,NaN
146,175,E,0,NaN,NaN,NaN,NaN,NaN,0.0,NaN,...,1.0,3,1,5.0,NaN,NaN,NaN,0.507393,0.480677,1.0
157,188,P,19391208,0,0.0,NaN,AGO,PRT,NaN,NaN,...,NaN,2,1,NaN,5.0,5.0,5.0,0.751332,0.724385,NaN


In [29]:
entities.date_of_birth.value_counts().head()

0           4639
19820101      23
19740101      17
19800217      16
19800505      16
Name: date_of_birth, dtype: int64

There shouldn't be people that have a birth date 0. Could these entities be companies? Or should we flag them as a higher risk?

In [45]:
#entities.B_age_risk.value_counts().head()

In [30]:
entities.sort_values(by = "date_of_birth").head(10)

,entity_number,entity_type,date_of_birth,qualifications_code,occupation_activity_code,occupation_code,nationality_code,country_of_residence_code,economic_activity_main_code,economic_activity_sec_code,...,country_of_residence_risk,cutoff_cl,economic_activity_code_risk,nationality_risk,occupation_risk,qualifications_risk,reg,score,seg,society_type_risk
48223,50180,P,-19790222,B,11.0,NaN,CPV,CPV,NaN,NaN,...,1,0,NaN,1.0,3.0,1.0,0.131495,0.164413,P,NaN
21699,23117,P,-19770215,0,0.0,NaN,CPV,CPV,NaN,NaN,...,1,0,NaN,1.0,5.0,5.0,0.322092,0.345001,P,NaN
26445,27984,P,-19720928,4,13.0,NaN,SEN,SEN,NaN,NaN,...,3,0,NaN,3.0,2.0,4.0,0.449028,0.412502,P,NaN
32767,34432,E,0,NaN,NaN,NaN,NaN,NaN,0.0,NaN,...,3,1,5.0,NaN,NaN,NaN,0.692417,0.586571,E,3.0
35929,37666,E,0,NaN,NaN,NaN,NaN,CPV,6810.0,NaN,...,1,0,5.0,NaN,NaN,NaN,0.384834,0.357703,E,1.0
35935,37672,E,0,NaN,NaN,NaN,NaN,CPV,0.0,NaN,...,1,1,5.0,NaN,NaN,NaN,0.633693,0.550337,E,4.0
35941,37678,E,0,NaN,NaN,NaN,NaN,CPV,8890.0,NaN,...,1,1,4.0,NaN,NaN,NaN,0.480133,0.440667,E,4.0
35950,37688,E,0,NaN,NaN,NaN,NaN,CPV,3601.0,8890.0,...,1,0,4.0,NaN,NaN,NaN,0.288625,0.285389,E,1.0
35959,37697,E,0,NaN,NaN,NaN,NaN,CPV,8890.0,NaN,...,1,1,4.0,NaN,NaN,NaN,0.537485,0.478022,E,4.0
35962,37700,E,0,NaN,NaN,NaN,NaN,CPV,4752.0,NaN,...,1,0,3.0,NaN,NaN,NaN,0.383925,0.333252,E,4.0


In [47]:
entities.loc[(entities['date_of_birth'] == 0)].shape

(4639, 35)

In [48]:
entities.loc[(entities['date_of_birth'] == 0) & (entities["entity_type"] != "P")].shape

(4530, 35)

In [51]:
entities.loc[(entities['date_of_birth'] == 0) & (entities["entity_type"] == "P")].shape

(109, 35)

Why are these birth dates negative?

### Correct

In [31]:
entities.loc[(entities['date_of_birth'] < 0)]

,entity_number,entity_type,date_of_birth,qualifications_code,occupation_activity_code,occupation_code,nationality_code,country_of_residence_code,economic_activity_main_code,economic_activity_sec_code,...,country_of_residence_risk,cutoff_cl,economic_activity_code_risk,nationality_risk,occupation_risk,qualifications_risk,reg,score,seg,society_type_risk
21699,23117,P,-19770215,0,0.0,NaN,CPV,CPV,NaN,NaN,...,1,0,NaN,1.0,5.0,5.0,0.322092,0.345001,P,NaN
26445,27984,P,-19720928,4,13.0,NaN,SEN,SEN,NaN,NaN,...,3,0,NaN,3.0,2.0,4.0,0.449028,0.412502,P,NaN
48223,50180,P,-19790222,B,11.0,NaN,CPV,CPV,NaN,NaN,...,1,0,NaN,1.0,3.0,1.0,0.131495,0.164413,P,NaN


In [33]:
entities['date_of_birth'] = abs(entities['date_of_birth'])

In [34]:
entities

,entity_number,entity_type,date_of_birth,qualifications_code,occupation_activity_code,occupation_code,nationality_code,country_of_residence_code,economic_activity_main_code,economic_activity_sec_code,...,country_of_residence_risk,cutoff_cl,economic_activity_code_risk,nationality_risk,occupation_risk,qualifications_risk,reg,score,seg,society_type_risk
0,2,P,19460908,4,90.0,NaN,CPV,CPV,NaN,NaN,...,1,0,NaN,1.0,4.0,4.0,0.241569,0.290509,P,NaN
1,3,P,19391124,4,10.0,NaN,PRT,PRT,NaN,NaN,...,2,0,NaN,2.0,4.0,4.0,0.411046,0.423553,P,NaN
2,4,P,19460101,2,13.0,NaN,PRT,PRT,NaN,NaN,...,2,0,NaN,2.0,2.0,2.0,0.250000,0.268998,P,NaN
3,5,P,19760709,1,90.0,NaN,CPV,CPV,NaN,NaN,...,1,0,NaN,1.0,4.0,3.0,0.226793,0.269547,P,NaN
4,6,P,19700328,3,13.0,NaN,CPV,CPV,NaN,NaN,...,1,0,NaN,1.0,2.0,3.0,0.095298,0.145518,P,NaN
5,7,P,19770506,2,13.0,NaN,CPV,CPV,NaN,NaN,...,1,0,NaN,1.0,2.0,2.0,0.080523,0.135954,P,NaN
6,8,P,19680330,4,13.0,NaN,CPV,CPV,NaN,NaN,...,1,0,NaN,1.0,2.0,4.0,0.110073,0.166481,P,NaN
7,9,P,19730620,2,13.0,NaN,CPV,CPV,NaN,NaN,...,1,0,NaN,1.0,2.0,2.0,0.080523,0.135954,P,NaN
8,10,P,19710222,4,13.0,NaN,CPV,CPV,NaN,NaN,...,1,0,NaN,1.0,2.0,4.0,0.110073,0.166481,P,NaN
9,11,P,19720119,4,10.0,NaN,CPV,CPV,NaN,NaN,...,1,0,NaN,1.0,4.0,4.0,0.241569,0.290509,P,NaN


In [ ]:
df[df.neighborhood == 'Belém']

In [ ]:
df[(df.neighborhood == 'Belém') & (df.bedrooms > 3)]

In [77]:
no_birth = entities[(entities.entity_type == 'P') & (entities.date_of_birth ==0)]

In [79]:
no_birth.head()

,entity_number,entity_type,date_of_birth,qualifications_code,occupation_activity_code,occupation_code,nationality_code,country_of_residence_code,economic_activity_main_code,economic_activity_sec_code,...,country_of_residence_risk,cutoff_cl,economic_activity_code_risk,nationality_risk,occupation_risk,qualifications_risk,reg,score,seg,society_type_risk
250,296,P,0,NaN,NaN,NaN,NaN,CPV,NaN,NaN,...,1,0,NaN,3.0,3.0,3.0,0.334221,0.306411,P,NaN
8112,8806,P,0,NaN,NaN,NaN,NaN,CPV,NaN,8890.0,...,1,0,NaN,3.0,3.0,3.0,0.334221,0.306411,P,NaN
8681,9425,P,0,NaN,NaN,NaN,NaN,CPV,NaN,NaN,...,1,0,NaN,3.0,3.0,3.0,0.334221,0.306411,P,NaN
8761,9510,P,0,NaN,NaN,NaN,NaN,CPV,NaN,NaN,...,1,0,NaN,3.0,3.0,3.0,0.334221,0.306411,P,NaN
9532,10334,P,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,3,0,NaN,3.0,3.0,3.0,0.500000,0.429558,P,NaN


In [71]:
entities.drop(no_birth.index, axis = 0)

In [81]:
#df.query('neighborhood == "Belém"').head()

In [ ]:
df.drop(df.loc[:, 'B':'D'].columns, axis = 1)

In [52]:
entities.drop(entities.loc[(entities['date_of_birth'] == 0) & (entities["entity_type"] == "P")])

KeyError: "['entity_number' 'entity_type' 'date_of_birth' 'qualifications_code'\n 'occupation_activity_code' 'occupation_code' 'nationality_code'\n 'country_of_residence_code' 'economic_activity_main_code'\n 'economic_activity_sec_code' 'date_of_constitution' 'society_type_code'\n 'gender' 'civil_status' 'BC_reg' 'BC_score' 'B_age_risk'\n 'B_company_age_risk' 'B_country_of_residence_risk'\n 'B_economic_activity_code_risk' 'B_nationality_risk' 'B_occupation_risk'\n 'B_qualifications_risk' 'B_society_type_risk' 'age_risk'\n 'company_age_risk' 'country_of_residence_risk' 'cutoff_cl'\n 'economic_activity_code_risk' 'nationality_risk' 'occupation_risk'\n 'qualifications_risk' 'reg' 'score' 'seg' 'society_type_risk'] not found in axis"

### Variable Grouping

In [54]:
entities.nationality_code.value_counts(normalize = True)

CPV    0.738883
PRT    0.102400
GNB    0.032323
ITA    0.013512
FRA    0.012755
GBR    0.012442
ESP    0.010829
CHN    0.010566
USA    0.009035
SEN    0.008410
NLD    0.007011
NGA    0.006600
BRA    0.004625
DEU    0.002880
GIN    0.002502
CUB    0.002271
AGO    0.002222
BEL    0.001991
STP    0.001777
CHE    0.001745
IRL    0.001563
LUX    0.001284
GHA    0.001086
SWE    0.000938
GMB    0.000856
RUS    0.000757
IND    0.000708
MOZ    0.000494
CMR    0.000494
SLE    0.000477
         ...   
TUR    0.000033
GEO    0.000033
KOR    0.000033
RWA    0.000033
HND    0.000033
NZL    0.000033
SOM    0.000033
NAM    0.000033
COG    0.000033
SLV    0.000016
SGP    0.000016
CAF    0.000016
GTM    0.000016
YUG    0.000016
LVA    0.000016
DOM    0.000016
MYS    0.000016
SPM    0.000016
HKG    0.000016
BYS    0.000016
GNQ    0.000016
ARE    0.000016
ZWE    0.000016
NCL    0.000016
NER    0.000016
MDA    0.000016
GUF    0.000016
DZA    0.000016
MUS    0.000016
SYC    0.000016
Name: nationality_code, 

In [3]:
Europe = ["BIH", "BLR", "GIB", "MCO", "RUS", "SRB", "UKR", "YUG", "ALB", "JEY", "MKD", "BGR", "MNE", "LIE", "MDA", "CHE", "CYP", "CZE", "FRO", "ESP", "BYS", "EST", "GBR", "IMN","ALA", "SMR", "AND", "GGY", "GRC", "HRV", "SJM", "HUN", "ITA", "LTU", "LVA", "MLT", "LUX", "NLD", "POL", "PRT", "ROM", "ROU", "SVK", "SVN", "VAT", "AUT", "BEL", "DEU", "DNK", "FIN", "FRA", "IRL", "ISL", "NOR", "SWE"]
Asia = ["CHN", "KHM", "KOR", "LAO", "LKA", "MYS", "NPL", "PRK", "BGD", "MNG", "THA", "VNM", "MMR", "HKG", 'IND', 'TWN', 'JPN', 'SGP', 'MAC', 'BTN', 'IOT', 'MDV']
Africa = ['CAF', 'COD', 'TZA', 'DZA', 'MYT', 'EGY', 'ETH', 'GIN', 'KEN', 'LBR', 'LBY', 'ZAF', 'BDI', 'AGO', 'CPV', 'TGO', 'MWI', 'SYC', 'HVO', 'MRT', 'GNQ', 'BFA', 'MOZ', 'LSO', 'STP', 'MLI', 'GMB', 'BEN', 'RWA', 'MUS', 'ZMB', 'SLE', 'NGA', 'SDN', 'SOM', 'GAB', 'DJI', 'SSD', 'GNB', 'COM', 'TCD', 'TUN', 'ZWE', 'CMR', 'ESH', 'MAR', 'MDG', 'NAM', 'UGA', 'BWA', 'CIV', 'GHA', 'NER', 'SWZ', 'SEN' ]
North_Central_America =['CRI', 'GTM', 'HND', 'PAN', 'NIC', 'GRL', 'MEX', 'CAN', 'USA', "SPM", "BLZ"]
South_America = ['BOL', 'ECU', 'PRY', 'VEN', 'COL', 'PER', 'SLV', 'TUR', 'BRA', 'URY', 'ARG', 'CHL', 'GUY', 'SUR', 'GUF']
Middle_East = ['AFG', 'ARE', 'IRN', 'IRQ', 'JOR', 'KWT', 'LBN', 'PAK', 'PSE', 'QAT', 'SYR', 'YEM', 'ARM', 'AZE', 'KAZ', 'KGZ', 'UZB', 'GEO', 'ISR', 'SAU', 'BHR', 'ERI', 'OMN', 'TJK', 'TKM']
Carribean = ['ATG', 'BHS', 'BMU', 'BRB', 'CUB', 'CYM', 'DOM', 'GRD', 'HTI', 'JAM', 'KNA', 'PRI', 'TTO', 'VGB', 'CUW', 'REU', 'TCA', 'ABW', 'VIR', 'AIA', 'DMA', 'LCA', 'MSR', 'SHN', 'VCT', 'ANT', 'SXM', 'ASC', 'BES', 'BLM', 'MTQ', 'TAA', 'MAF']
Oceania = []

SyntaxError: invalid syntax (<ipython-input-3-f6b9da406b10>, line 3)

### Missing values

In [ ]:
n_records = len(entities)
def missing_values_df(df):
    for column in df:
        print("{} | {} | {}".format(
            column, len(df[df[column].isnull()]) / (1.0*n_records), df[column].dtype
        ))

missing_values_df(entities)

There are several variables with repeated values. The highest are `economic_activity_main_code`, `economic_activity_sec_code`, `B_company_age_risk`, `B_economic_activity_code_risk`, `company_age_risk`, `economic_activity_code_risk`, `society_type_risk` with a values around 0.93.

In [ ]:
def cardinality_categorical(df):
    n_records = len(df)
    for column in df.select_dtypes([object]):
        print("{} | uniques/records: {:.3f} | Minimum observations: {:.3f}".format(
            column, 
            len(df[column].unique())/n_records,
            df[column].value_counts().min()
        ))        

cardinality_categorical(entities)

### Grouping Variables

In [ ]:
entities['score'].plot.hist();

In [ ]:
entities['score'].plot.kde();

In [ ]:
def numerical_variable_distribution(df, col):
    df[col].plot.kde()
    plt.xlabel('Variable "{}" Distribution'.format(col))
    plt.show()

In [ ]:
numerical_columns = entities.select_dtypes(include=[np.number]).columns.tolist()

In [ ]:
from ipywidgets import interact, fixed

In [ ]:
interact(numerical_variable_distribution, 
         col=numerical_columns, df=fixed(entities));

In [ ]:
from scipy import stats

In [ ]:
def numerical_variable_normality(col):
    stats.probplot(entities[col], plot=plt)
    plt.xlabel('Probability plot for variable {}'.format(col))
    plt.show()

In [ ]:
interact(numerical_variable_normality, col=numerical_columns);

In [ ]:
for num_col in numerical_columns:
    _, pval = stats.normaltest(entities[num_col][entities[num_col].notnull()])
    if(pval < 0.05):
        print("Column {} doesn't follow a normal distribution".format(num_col))

In [ ]:
def categorical_variable_distribution(col):
    entities[col].value_counts(ascending=True,normalize=True).tail(20).plot.barh()
    plt.show()

In [ ]:
categorical_columns = entities.select_dtypes(
    ['object', 'category']).columns.tolist()

In [ ]:
categorical_columns

In [ ]:
interact(categorical_variable_distribution, col=categorical_columns);

In [ ]:
def pivot_count(df, rows, columns):
    df_pivot = df.pivot_table(values="date_of_constitution", # could be any column, since we are just counting rows 
                              index=rows, 
                              columns=columns, 
                              aggfunc=np.size
                             ).dropna(axis=0, how='all')
    return df_pivot

In [ ]:
pivot1 = pivot_count(entities, "civil_status","nationality_code")
pivot1

In [ ]:
def heatmap_count_types(df, col1, col2):
    pivot_table = pivot_count(df,col1, col2)
    sns.heatmap(pivot_table, annot=True, fmt='g')
    plt.ylabel(col1)
    plt.xlabel(col2)
    plt.show()

In [ ]:
heatmap_count_types(entities, "gender","nationality_code")

In [ ]:
entities.dtypes

In [ ]:
interact(heatmap_count_types, col1=entities.columns.drop(["entity_number", "seg"]),
                                 col2=entities.columns.drop(["entity_number", "seg"]),
                                 df=fixed(entities));